In [20]:
import pandas as pd
#from keras.models import load_model

#from utils.transformers import smile2unfold
#from utils.transformers import trim_padding
#from utils.transformers import pad_inputs
#from utils.transformers import gene2ordinal
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import pickle as pkl
import importlib

import utils.transformers as transformers
importlib.reload(transformers)


import matplotlib.pyplot as plt

Traceback (most recent call last):
  File "/miniconda/lib/python3.6/site-packages/rdkit/Chem/PandasTools.py", line 130, in <module>
    if 'display.width' in pd.core.config._registered_options:
AttributeError: module 'pandas.core' has no attribute 'config'


In [2]:
DB_df = pd.read_pickle('validation_data/drugbank_fingerprinted_with_drug_strings.pkl')

In [3]:
DB_df

,drug_cas,drug_id,drug_name,target_actions,target_amino,target_gene,target_id,target_name,target_organism,pubchem_id,pubchem_name,target_gene_fingerprint,drug_fingerprint
31,128270-60-0,DB00006,Bivalirudin,[inhibitor],>lcl|BSEQ0016004|Prothrombin\nMAHVRGLQLPGCLALA...,>lcl|BSEQ0016005|Prothrombin (F2)\nATGGCGCACGT...,BE0000048,Prothrombin,Human,16129704,Bivalirudin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2097 2246728737 3542456614 2245384272 15065...
53,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0036957|Lutropin-choriogonadotropic h...,>lcl|BSEQ0010174|Lutropin-choriogonadotropic h...,BE0000134,Lutropin-choriogonadotropic hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,785 2246728737 3537119515 2245273601 242354...
54,65807-02-5,DB00014,Goserelin,[agonist],>lcl|BSEQ0000405|Gonadotropin-releasing hormon...,>lcl|BSEQ0018926|Gonadotropin-releasing hormon...,BE0000203,Gonadotropin-releasing hormone receptor,Human,5311128,goserelin,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,785 2246728737 3537119515 2245273601 242354...
131,16679-58-6,DB00035,Desmopressin,[agonist],>lcl|BSEQ0000583|Vasopressin V2 receptor\nMLMA...,>lcl|BSEQ0010065|Vasopressin V2 receptor (AVPR...,BE0000293,Vasopressin V2 receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,6012 2968968094 2142032900 2968968094 21170...
132,16679-58-6,DB00035,Desmopressin,NaN,>lcl|BSEQ0009979|Vasopressin V1a receptor\nMRL...,>lcl|BSEQ0009980|Vasopressin V1a receptor (AVP...,BE0000165,Vasopressin V1a receptor,Human,16051933,DESMOPRESSIN,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,6012 2968968094 2142032900 2968968094 21170...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,782500-75-8,DB09043,Albiglutide,[agonist],>lcl|BSEQ0010605|Glucagon-like peptide 1 recep...,>lcl|BSEQ0010606|Glucagon-like peptide 1 recep...,BE0000857,Glucagon-like peptide 1 receptor,Human,122173812,Albiglutide,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,3598 2246728737 3542456614 2245384272 15065...
457,8063-17-0,DB09363,Rauwolfia serpentina root,[inhibitor],>lcl|BSEQ0000236|Synaptic vesicular amine tran...,>lcl|BSEQ0009946|Synaptic vesicular amine tran...,BE0000118,Synaptic vesicular amine transporter,Human,441975,Ajmalicine,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,3745 2246728737 1858577693 2976033787 26408...
460,844439-96-9,DB09564,Insulin Degludec,[ligand],>lcl|BSEQ0036940|Insulin receptor\nMATGGRRGAAA...,>lcl|BSEQ0020443|Insulin receptor (INSR)\nATGG...,BE0000033,Insulin receptor,Human,118984462,Insulin degludec,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,1133 2246728737 3542456614 2245384272 15065...
466,NaN,DB11365,Sennosides,[inhibitor],>lcl|BSEQ0012880|Reverse transcriptase/RNaseH\...,NaN,BE0004136,Reverse transcriptase/RNaseH,Human immunodeficiency virus 1,656822,sennoside,NaN,409 3218693969 98513984 3218693969 95122607...


In [5]:
nonbinding_df = pd.read_pickle('validation_data/nonbinding_pairs_processed.pkl')

In [6]:
nonbinding_df

,drug_id,fingerprint,target_id,gene,amino
0,DB04481,5628 2245384272 4022716898 2245273601 17354...,BE0002266,>lcl|BSEQ0011632|N-acetylglucosamine-1-phospho...,>lcl|BSEQ0037182|N-acetylglucosamine-1-phospho...
1,DB04481,5628 2245384272 4022716898 2245273601 17354...,BE0000206,>lcl|BSEQ0010005|Retinoic acid receptor beta (...,>lcl|BSEQ0000411|Retinoic acid receptor beta\n...
2,DB04481,5628 2245384272 4022716898 2245273601 17354...,BE0002401,>lcl|BSEQ0016780|P2Y purinoceptor 2 (P2RY2)\nA...,>lcl|BSEQ0016779|P2Y purinoceptor 2\nMAADLGPWN...
3,DB04481,5628 2245384272 4022716898 2245273601 17354...,BE0004180,>lcl|BSEQ0049969|Penicillin-binding protein 2 ...,>lcl|BSEQ0007950|Penicillin-binding protein 2\...
4,DB04481,5628 2245384272 4022716898 2245273601 17354...,BE0002840,>lcl|BSEQ0016907|Estradiol 17-beta-dehydrogena...,>lcl|BSEQ0016906|Estradiol 17-beta-dehydrogena...
...,...,...,...,...,...
775178,DB01105,1532 2246728737 3537119515 2245273601 24235...,BE0004557,>lcl|BSEQ0013186|Neuronal calcium sensor 1 (NC...,>lcl|BSEQ0009478|Neuronal calcium sensor 1\nMG...
775179,DB02056,5161 2246728737 3542456614 2245384272 11731...,BE0002410,>lcl|BSEQ0011725|Adhesion G-protein coupled re...,>lcl|BSEQ0037203|Adhesion G-protein coupled re...
775180,DB03453,2999 2246728737 3975275337 864674487 215493...,BE0002410,>lcl|BSEQ0011725|Adhesion G-protein coupled re...,>lcl|BSEQ0037203|Adhesion G-protein coupled re...
775181,DB02056,5161 2246728737 3542456614 2245384272 11731...,BE0004557,>lcl|BSEQ0013186|Neuronal calcium sensor 1 (NC...,>lcl|BSEQ0009478|Neuronal calcium sensor 1\nMG...


In [7]:
with open('validation_data/Corona_Gene_Embeddings.pkl', 'rb') as f:
    corona_targets_df = pkl.load(f)

In [8]:
corona_targets_df

,Gene,Gene_Name,Entrez_ID,Target_DB_ID,AA_Sequence,Gene_Sequence,Gene_FingerPrint,Target_EMbedding
0,ACADM,"Medium-chain specific acyl-CoA dehydrogenase, ...",34.0,BE0001745,>lcl|BSEQ0003460|Medium-chain specific acyl-Co...,>lcl|BSEQ0021302|Medium-chain specific acyl-Co...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.0008826057 -0.0018619265 0.0005886211 0.0001...
1,GGCX,Tissue factor,2677.0,BE0001178,>lcl|BSEQ0002343|Tissue factor\nMETPAWPRVPRPET...,>lcl|BSEQ0021656|Tissue factor (F3)\nATGGAGACC...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.0004959933 -0.0015191967 0.0004373464 -0.000...
2,ATP6V1A,V-type proton ATPase catalytic subunit A,523.0,BE0003569,>lcl|BSEQ0006875|V-type proton ATPase catalyti...,>lcl|BSEQ0012482|V-type proton ATPase catalyti...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.00046752422 -0.0017994677 0.0003406349 -0.00...
3,TARS2,"Threonine--tRNA ligase, mitochondrial",80222.0,BE0002317,">lcl|BSEQ0004550|Threonine--tRNA ligase, mitoc...",>lcl|BSEQ0004549|2157 bp\nATGGCCCTGTATCAGAGGTG...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.0007386954 -0.0014467473 0.00034487134 -0.00...
4,AASS,"Glutamate receptor ionotropic, kainate 3",10157.0,BE0000832,>lcl|BSEQ0037089|Glutamate receptor ionotropic...,>lcl|BSEQ0010587|Glutamate receptor ionotropic...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.00022724312 -0.00087777985 -3.299319e-05 -0....
5,CSNK2A2,Casein kinase II subunit alpha',1459.0,BE0003984,>lcl|BSEQ0012773|Casein kinase II subunit alph...,>lcl|BSEQ0012774|Casein kinase II subunit alph...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.00063122425 -0.001525819 0.000564763 -0.0007...
6,POLA1,Purine nucleoside phosphorylase,5422.0,BE0000250,>lcl|BSEQ0010033|Purine nucleoside phosphoryla...,>lcl|BSEQ0010034|Purine nucleoside phosphoryla...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.00042263846 -0.0014812584 0.00028048162 -3.6...
7,POLA2,"6-phosphogluconate dehydrogenase, decarboxylating",23649.0,BE0003559,>lcl|BSEQ0006857|6-phosphogluconate dehydrogen...,>lcl|BSEQ0020701|6-phosphogluconate dehydrogen...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.0004616005 -0.0010432433 0.00059503893 -0.00...
8,GFER,NADH dehydrogenase [ubiquinone] 1 alpha subcom...,2671.0,BE0000305,>lcl|BSEQ0000607|NADH dehydrogenase [ubiquinon...,>lcl|BSEQ0010072|NADH dehydrogenase [ubiquinon...,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,0.0001736389 -0.0013006188 -1.8839728e-05 -0.0...
9,BCKDK,Cag alpha,10295.0,BE0002850,>lcl|BSEQ0022302|Cag alpha\nMTEDRLSAEDKKFLEVER...,>lcl|BSEQ0005579|993 bp\nCTACCTATGTTTGATATAAAA...,[0.0001882191 0.00042436537 -0.0014462507 -0.0...,-0.0011031309 -0.001748469 0.00023615667 6.136...


In [9]:
corona_target_ids = corona_targets_df['Target_DB_ID'].tolist()

In [10]:
to_drop_rows = []

for index, row in DB_df.iterrows():
    if row['target_id'] not in corona_target_ids:
        to_drop_rows.append(index)
        
DB_df = DB_df.drop(to_drop_rows)

In [11]:
to_drop_rows = []

for index, row in nonbinding_df.iterrows():
    if row['target_id'] not in corona_target_ids:
        to_drop_rows.append(index)
        
nonbinding_df = nonbinding_df.drop(to_drop_rows)

# Binding Data Processing

## Drug - Preprocessing

In [12]:
DB_df = DB_df[~ DB_df['pubchem_id'].isna()]

In [13]:
DB_df = DB_df.rename(columns={"drug_fingerprint": "fingerprint"})

In [14]:
DB_df = DB_df[~ DB_df['fingerprint'].isna()]

In [15]:
DB_df = DB_df.dropna(subset=['fingerprint'])

In [16]:
DB_df['fingerprint'] = DB_df['fingerprint'].apply(lambda x: ''.join([i for i in x.values]))

In [21]:
DB_df = transformers.DDR_df_pad_fingerprint(DB_df)

In [22]:
DB_df

,drug_cas,drug_id,drug_name,target_actions,target_amino,target_gene,target_id,target_name,target_organism,pubchem_id,pubchem_name,target_gene_fingerprint,fingerprint,drug_fingerprint_encoded,drug_fingerprint_encoded_len,drug_fingerprint_encoded_mismatch,drug_fingerprint_encoded_padded
535,54-47-7,DB00114,Pyridoxal Phosphate,[cofactor],>lcl|BSEQ0022280|Alanine-glyoxylate aminotrans...,>lcl|BSEQ0004265|813 bp\nATGGCACTGAGGGGCCCAGGG...,BE0002174,Alanine-glyoxylate aminotransferase homolog,Human,1051,pyridoxal phosphate,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2246728737 422715066 3217380708 4033380444 204...,"[2, 2, 4, 6, 7, 2, 8, 7, 3, 7, 4, 2, 2, 7, 1, ...",309,1638,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
640,50-81-7,DB00126,Vitamin C,NaN,>lcl|BSEQ0004363|Transmembrane prolyl 4-hydrox...,>lcl|BSEQ0011604|Transmembrane prolyl 4-hydrox...,BE0002223,Transmembrane prolyl 4-hydroxylase,Human,54670067,l-ascorbic acid,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2245384272 4022716898 2245273601 3081284555 29...,"[2, 2, 4, 5, 3, 8, 4, 2, 7, 2, 4, 0, 2, 2, 7, ...",228,1719,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
832,56-86-0,DB00142,Glutamic Acid,NaN,>lcl|BSEQ0037089|Glutamate receptor ionotropic...,>lcl|BSEQ0010587|Glutamate receptor ionotropic...,BE0000832,"Glutamate receptor ionotropic, kainate 3",Human,33032,L-glutamic acid,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2245384272 1583799011 2245384272 1167322652 22...,"[2, 2, 4, 5, 3, 8, 4, 2, 7, 2, 1, 5, 8, 3, 7, ...",195,1752,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
881,1446756-47-3,DB00144,Phosphatidyl serine,NaN,>lcl|BSEQ0000377|Scavenger receptor class B me...,>lcl|BSEQ0009993|Scavenger receptor class B me...,BE0000189,Scavenger receptor class B member 1,Human,6323481,phosphatidyl-L-serine,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2246728737 3542456614 2245384272 1173125914 22...,"[2, 2, 4, 6, 7, 2, 8, 7, 3, 7, 3, 5, 4, 2, 4, ...",486,1461,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
920,57-00-1,DB00148,Creatine,[product of],>lcl|BSEQ0001132|Guanidinoacetate N-methyltran...,>lcl|BSEQ0016138|Guanidinoacetate N-methyltran...,BE0000567,Guanidinoacetate N-methyltransferase,Human,586,creatine,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2246728737 3818546315 848128881 3136829774 224...,"[2, 2, 4, 6, 7, 2, 8, 7, 3, 7, 3, 8, 1, 8, 5, ...",174,1773,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15514,10103-46-5,DB11348,Calcium Phosphate,[ligand],>lcl|BSEQ0009478|Neuronal calcium sensor 1\nMG...,>lcl|BSEQ0013186|Neuronal calcium sensor 1 (NC...,BE0004557,Neuronal calcium sensor 1,Human,24456,Calcium phosphate,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,864942795 2235914803 984189120 4036290423 8649...,"[8, 6, 4, 9, 4, 2, 7, 9, 5, 2, 2, 3, 5, 9, 1, ...",220,1727,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
15566,726169-73-9,DB11830,Mocetinostat,NaN,>lcl|BSEQ0004647|Histone deacetylase 2\nMAYSQG...,>lcl|BSEQ0016767|Histone deacetylase 2 (HDAC2)...,BE0002366,Histone deacetylase 2,Human,9865515,Mocetinostat,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,3218693969 98513984 3218693969 98513984 321869...,"[3, 2, 1, 8, 6, 9, 3, 9, 6, 9, 9, 8, 5, 1, 3, ...",580,1367,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
56,188627-80-7,DB00063,Eptifibatide,NaN,>lcl|BSEQ0002297|Integrin beta-3\nMRARPRPRPLWA...,>lcl|BSEQ0021847|Integrin beta-3 (ITGB3)\nATGC...,BE0001155,Integrin beta-3,Human,448812,Eptifibatide,[1.2739266e-05 -2.7748918e-05 0.00095751503 0....,2968968094 2142032900 2968968094 2117068077 29...,"[2, 9, 6, 8, 9, 6, 8, 0, 9, 4, 2, 1, 4, 2, 0, ...",1105,842,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
168,NaN,DB05033,INCB7839,NaN,>lcl|BSEQ0006659|Disintegrin and metalloprotei...,>lcl|BSEQ0017094|Disintegrin and metalloprotei...,BE0003412,Disintegrin and metalloproteinase domain-conta...,Human,16070111,Aderbasib,[1.27392

## Target - Preprcessing

In [23]:
to_drop_rows = []

for index, row in DB_df.iterrows():
    if type(row['target_amino']) is float or type(row['target_gene']) is float:
        to_drop_rows.append(index)

In [24]:
len(to_drop_rows)

0

In [25]:
DB_df = DB_df.drop(to_drop_rows)

In [71]:
DB_df['target_gene_code'] = DB_df['target_gene'].apply(lambda x: ''.join(list(x.split('\n')[1:])))

In [26]:
DB_df['amino'] = DB_df['target_amino'].apply(lambda x: ''.join(x.split('\n')[1:]))

In [27]:
DB_df['gene'] = DB_df['target_gene'].apply(lambda x: ''.join(x.split('\n')[1:]))

In [30]:
DB_df = transformers.DDR_df_pad_gene(DB_df)

In [31]:
DB_df = transformers.DDR_df_pad_amino(DB_df)

In [32]:
DB_df

,drug_cas,drug_id,drug_name,target_actions,target_amino,target_gene,target_id,target_name,target_organism,pubchem_id,...,amino,gene,target_gene_encoded,target_gene_encoded_len,target_gene_encoded_mismatch,target_gene_encoded_padded,target_amino_encoded,target_amino_encoded_len,target_amino_encoded_mismatch,target_amino_encoded_padded
535,54-47-7,DB00114,Pyridoxal Phosphate,[cofactor],>lcl|BSEQ0022280|Alanine-glyoxylate aminotrans...,>lcl|BSEQ0004265|813 bp\nATGGCACTGAGGGGCCCAGGG...,BE0002174,Alanine-glyoxylate aminotransferase homolog,Human,1051,...,MALRGPGLERPPLTCPELRQEQCCGGCQVGGAGPERRGGLGASTLS...,ATGGCACTGAGGGGCCCAGGGCTGGAGAGACCACCCCTCACCTGTC...,"[0, 3, 2, 2, 1, 0, 1, 3, 2, 0, 2, 2, 2, 2, 1, ...",813,3783,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 0, 9, 14, 5, 12, 5, 9, 3, 14, 12, 12, 9, ...",270,1261,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
640,50-81-7,DB00126,Vitamin C,NaN,>lcl|BSEQ0004363|Transmembrane prolyl 4-hydrox...,>lcl|BSEQ0011604|Transmembrane prolyl 4-hydrox...,BE0002223,Transmembrane prolyl 4-hydroxylase,Human,54670067,...,MAAAAVTGQRPETAAAEEASRPQWAPPDHCQAQAAAGLGDGEDAPV...,ATGGCGGCAGCGGCGGTGACAGGCCAGCGGCCTGAGACCGCGGCGG...,"[0, 3, 2, 2, 1, 2, 2, 1, 0, 2, 1, 2, 2, 1, 2, ...",1692,2904,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 0, 0, 0, 0, 18, 16, 5, 13, 14, 12, 3, 16,...",502,1029,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
832,56-86-0,DB00142,Glutamic Acid,NaN,>lcl|BSEQ0037089|Glutamate receptor ionotropic...,>lcl|BSEQ0010587|Glutamate receptor ionotropic...,BE0000832,"Glutamate receptor ionotropic, kainate 3",Human,33032,...,MTAPWRRLRSLVWEYWAGLLVCAFWIPDSRGMPHVIRIGGIFEYAD...,ATGACCGCTCCCTGGCGGCGCCTCCGGAGTCTGGTTTGGGAATACT...,"[0, 3, 2, 0, 1, 1, 2, 1, 3, 1, 1, 1, 3, 2, 2, ...",2760,1836,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 16, 0, 12, 19, 14, 14, 9, 14, 15, 9, 18, ...",919,612,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
881,1446756-47-3,DB00144,Phosphatidyl serine,NaN,>lcl|BSEQ0000377|Scavenger receptor class B me...,>lcl|BSEQ0009993|Scavenger receptor class B me...,BE0000189,Scavenger receptor class B member 1,Human,6323481,...,MGCSAKARWAAGALGVAGLLCAVLGAVMIVMVPSLIKQQVLKNVRI...,ATGGGCTGCTCCGCCAAAGCGCGCTGGGCTGCCGGGGCGCTGGGCG...,"[0, 3, 2, 2, 2, 1, 3, 2, 1, 3, 1, 1, 2, 1, 1, ...",1521,3075,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 5, 1, 15, 0, 8, 0, 14, 19, 0, 0, 5, 0, 9,...",552,979,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
920,57-00-1,DB00148,Creatine,[product of],>lcl|BSEQ0001132|Guanidinoacetate N-methyltran...,>lcl|BSEQ0016138|Guanidinoacetate N-methyltran...,BE0000567,Guanidinoacetate N-methyltransferase,Human,586,...,MSAPSATPIFAPGENCSPAWGAAPAAYDAADTHLRILGKPVMERWE...,ATGAGCGCCCCCAGCGCGACCCCCATCTTCGCGCCCGGCGAGAACT...,"[0, 3, 2, 0, 2, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1, ...",711,3885,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 15, 0, 12, 15, 0, 16, 12, 7, 4, 0, 12, 5,...",236,1295,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15514,10103-46-5,DB11348,Calcium Phosphate,[ligand],>lcl|BSEQ0009478|Neuronal calcium sensor 1\nMG...,>lcl|BSEQ0013186|Neuronal calcium sensor 1 (NC...,BE0004557,Neuronal calcium sensor 1,Human,24456,...,MGKSNSKLKPEVVEELTRKTYFTEKEVQQWYKGFIKDCPSGQLDAA...,ATGGCAACGATTACCGAGAAGGAGGTCCAGCAGTGGTACAAAGGCT...,"[0, 3, 2, 2, 1, 0, 0, 1, 2, 0, 3, 3, 0, 1, 1, ...",519,4077,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[10, 5, 8, 15, 11, 15, 8, 9, 8, 12, 3, 18, 18,...",190,1341,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
15566,726169-73-9,DB11830,Mocetinostat,NaN,>lcl|BSEQ0004647|Histone deacetylase 2\nMAYSQG...,>lcl|BSEQ0016767|Histone deacetylase 2 (HDAC2)...,BE0002366,Histone deacetylase 2,Human,9865515,...,MAYSQGGGKKKVCYYYDGDIGNYYYGQGHPMKPHRIRMTHNLLLNY...,ATGGCGTACAGTCAAGGAGGCGGCAAAAAAAAAGTCTGCTACTACT...,"[0, 3, 2, 2, 1, 2, 3, 0, 1, 0, 2, 3, 1, 0, 0, ...",1467,3129,"[-1, -1, -1, -1, -

# Final Preprocessing

In [47]:
target_length = 30723 # This should be the models input dimension

In [67]:
X_0 = np.array(DB_df['target_gene_encoded_padded'].to_list())

In [68]:
X_1 = np.array(DB_df['drug_fingerprint_encoded_padded'].to_list())

In [70]:
X_0.shape,X_1.shape

((156, 4596), (156, 1947))

In [61]:
X_1_subset = np.vectorize(lambda x: int(x))(X_1)
X_0_subset = np.vectorize(lambda x: int(x))(X_0)

In [51]:
X_0_subset = transformers.trim_padding([X_0_subset], target_length = int((target_length + 1)/2))[0]
X_1_subset = transformers.trim_padding([X_1_subset], target_length = int(target_length/2))[0]

-5383
9979
-6707
8654


In [62]:
X = np.ones((X_0_subset.shape[0], X_0_subset.shape[1] + X_1_subset.shape[1]))

In [63]:
for i in range(len(X_0_subset)):
    X[i,:] = np.hstack((X_0_subset[i,:], X_1_subset[i,:])) + 1

In [64]:
X.shape

(156, 6543)

In [65]:
X_binding = X

In [ ]:
plt.hist(X_0_subset.mean(axis = 1))

In [ ]:
plt.hist(X_1_subset.mean(axis = 1))

In [ ]:
plt.hist(X.sum(axis = 1), bins = 100)

In [56]:
X = X[:, :target_length]

In [57]:
X_binding = X_binding[:, :target_length]

In [ ]:
plt.hist(X[0])

## Run Model

In [43]:
import tensorflow.keras
import tensorflow.keras as keras 
from tensorflow.keras.models import load_model
import pickle

In [44]:
#amino_model = load_model('data/models/pure_cnn.model')
amino_model = keras.models.load_model('models/CNN_GPU_VGG_80p.model')

In [66]:
%%time
Y = amino_model.predict(X_binding[..., np.newaxis])

ValueError: Error when checking input: expected input_4 to have shape (30723, 1) but got array with shape (6543, 1)

In [ ]:
sum(Y)/len(Y)

In [ ]:
positive_samples_Y = pd.DataFrame()

positive_samples_Y['Y'] = list(Y)

In [ ]:
#positive_samples_Y.to_pickle('positive_samples_Y.pkl')

In [ ]:
positive_samples_Y = pd.read_pickle('validation_data/positive_samples_Y.pkl')


In [ ]:
positive_samples_Y

In [ ]:
plt.hist(Y, bins = 100, log = True)
plt.xlabel('Predicted Score of Binding')
plt.ylabel('Number of Pairs')

In [ ]:
Y_binding_samples = []

for i in range(Y.shape[0]):
    Y_binding_samples.append(Y[i][0])

In [ ]:
Y_binding_samples

# Non-Binding Data Processing¶

## Drug Pre-Processing

In [ ]:
nonbinding_df['fingerprint'] = nonbinding_df['fingerprint'].apply(lambda x: ''.join([i for i in x.values]))

In [ ]:
nonbinding_df = transformers.DDR_df_pad_fingerprint(nonbinding_df)

## Target Pre-Processing

In [ ]:
nonbinding_df['amino'] = nonbinding_df['amino'].apply(lambda x: ''.join(x.split('\n')[1:]))

In [ ]:
nonbinding_df['gene'] = nonbinding_df['gene'].apply(lambda x: ''.join(x.split('\n')[1:]))

In [ ]:
nonbinding_df = transformers.DDR_df_pad_gene(nonbinding_df)

In [ ]:
nonbinding_df = transformers.DDR_df_pad_amino(nonbinding_df)

## Final Preprocessing

In [ ]:
X_0 = np.array(nonbinding_df['target_gene_encoded_padded'].to_list())

In [ ]:
X_1 = np.array(nonbinding_df['drug_fingerprint_encoded_padded'].to_list())

In [ ]:
X_0.shape,X_1.shape

In [ ]:
X_1_subset = np.vectorize(lambda x: int(x))(X_1)
X_0_subset = np.vectorize(lambda x: int(x))(X_0)

In [ ]:
X_0_subset = transformers.trim_padding([X_0_subset], target_length = int((target_length + 1)/2))[0]
X_1_subset = transformers.trim_padding([X_1_subset], target_length = int(target_length/2))[0]

In [ ]:
X = np.ones((X_1_subset.shape[0], X_0_subset.shape[1] + X_1_subset.shape[1]))

In [ ]:
X_0_subset.shape, X_1_subset.shape

In [ ]:
for i in range(len(X_1_subset)):
    X[i,:] = np.hstack((X_0_subset[i,:], X_1_subset[i,:])) + 1

In [ ]:
X.shape

In [ ]:
X_nonbinding = X

In [ ]:
X = X[:, :target_length]

In [ ]:
Y_nonbinding_samples = amino_model.predict(X[..., np.newaxis])

In [ ]:
plt.hist(Y_nonbinding_samples, bins = 100, log = True)
plt.xlabel('Predicted Score of Binding')
plt.ylabel('Number of Pairs')

In [ ]:
# Accuracy score
(len(Y_nonbinding_samples)-sum(Y_nonbinding_samples))/len(Y_nonbinding_samples)

In [ ]:
Y_nonbinding_samples_list = []

for i in range(Y_nonbinding_samples.shape[0]):
    Y_nonbinding_samples_list.append(Y_nonbinding_samples[i][0])

In [ ]:
negative_samples_Y = pd.DataFrame()
negative_samples_Y['Y'] = list(Y)
negative_samples_Y.to_pickle('negative_samples_Y.pkl')

# ROC Curve Plot

In [ ]:
import scikitplot as skplt
import matplotlib.pyplot as plt

y_true = list(Y_binding_samples) + list(Y_nonbinding_samples)
y_probas = list(np.ones(len(Y_binding_samples))) + list(np.zeros(len(Y_nonbinding_samples)))
skplt.metrics.plot_roc_curve(y_true, y_probas)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_probas = np.array(list(Y_binding_samples) + list(Y_nonbinding_samples_list))
y_true = np.array(list(np.ones(len(Y_binding_samples))) + list(np.zeros(len(Y_nonbinding_samples_list))))
false_positive_rate, true_positive_rate, threshold = roc_curve(y_true, y_probas)
print('roc_auc_score: ', roc_auc_score(y_true, y_probas))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(false_positive_rate, true_positive_rate)
plt.show()

# Retrain

In [ ]:
X_retrain_labels = np.zeros((X_binding.shape[0]+X_nonbinding.shape[0],X_binding.shape[1]+1))

In [ ]:
for i in range(X_binding.shape[0]):
    for j in range(X_binding.shape[1]):
        X_retrain_labels[i,j] = X_binding[i,j]

In [ ]:
for i in range(X_nonbinding.shape[0]):
    for j in range(X_nonbinding.shape[1]):
        X_retrain_labels[X_binding.shape[0]+i,j] = X_nonbinding[i,j]

In [ ]:
for i in range(X_binding.shape[0]+X_nonbinding.shape[0]):
    X_retrain_labels[i,X_binding.shape[1]] = y_true[i]

In [ ]:
X_retrain_labels.shape

In [ ]:
import keras
from keras.datasets import imdb

from tqdm.notebook import tqdm

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split



from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv1D, Flatten, MaxPooling1D,\
                        AveragePooling1D, Concatenate, LeakyReLU, Embedding, GlobalMaxPooling1D
from keras.utils.vis_utils import plot_model

In [ ]:
hidden_dims = 768
hidden_dims_2 = 256
hidden_dims_3 = 128
batch_size = 2639
epochs = 40
model_optimizer = keras.optimizers.Nadam(lr=0.1, beta_1=0.9, beta_2=0.999)

model = Sequential()

#model.add(Dropout(0.2))
model.add(Conv1D(4096,
                 2048,
                 strides=320,
                 activation='relu'))


model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(hidden_dims))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(hidden_dims_2))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(hidden_dims_3))
model.add(Dropout(0.3))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=model_optimizer,
              metrics=['binary_accuracy'])

In [ ]:
class_label = X_retrain_labels[:, -1] # for last column
dataset = X_retrain_labels[:, :-1] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, class_label, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train[..., np.newaxis], y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test[..., np.newaxis], y_test))